In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from shapely.wkt import loads
from shapely import wkt
import shapely.geometry
from shapely.geometry import Polygon
from IPython.display import display
pd.options.display.max_columns = None
import contextily as cx
import math
import os
import sys

#### Data Paths

In [2]:
input_path = "./../../data/output-data/dish/denver/final-impacts/"
gis_path = "./../../data/input-data/dish/gis/"
output_path = "./../../data/output-data/dish/denver/recommendations/crossed-feeders/"

#### Load Cell Data - Only for plotting purposes

In [3]:
def polysTriangle(lat, lon, bearing, hbeam):
	"""
	Function creates polygons for all cells, if "H_BEAM" = 360 then create circle
	"""
	R = 6378.1
	lat1 = math.radians(lat)
	lon1 = math.radians(lon)

	# Size cell based on 'tech'
	d = 0.5

	# Create LAT/LON points
	lat2 = math.asin(math.sin(lat1)*math.cos(d/R) +
					math.cos(lat1)*math.sin(d/R)*math.cos(math.radians(bearing - (hbeam / 2))))

	lon2 = lon1 + math.atan2(math.sin(math.radians(bearing - (hbeam / 2)))*math.sin(d/R)*math.cos(lat1),
					math.cos(d/R)-math.sin(lat1)*math.sin(lat2))

	lat3 = math.asin(math.sin(lat1)*math.cos(d/R) +
					math.cos(lat1)*math.sin(d/R)*math.cos(math.radians(bearing - (3 * hbeam / 8))))

	lon3 = lon1 + math.atan2(math.sin(math.radians(bearing - (3 * hbeam / 8)))*math.sin(d/R)*math.cos(lat1),
					math.cos(d/R)-math.sin(lat1)*math.sin(lat3))

	lat4 = math.asin(math.sin(lat1)*math.cos(d/R) +
					math.cos(lat1)*math.sin(d/R)*math.cos(math.radians(bearing - (hbeam / 4))))

	lon4 = lon1 + math.atan2(math.sin(math.radians(bearing - (hbeam / 4)))*math.sin(d/R)*math.cos(lat1),
					math.cos(d/R)-math.sin(lat1)*math.sin(lat4))

	lat5 = math.asin(math.sin(lat1)*math.cos(d/R) +
					math.cos(lat1)*math.sin(d/R)*math.cos(math.radians(bearing)))

	lon5 = lon1 + math.atan2(math.sin(math.radians(bearing))*math.sin(d/R)*math.cos(lat1),
					math.cos(d/R)-math.sin(lat1)*math.sin(lat5))

	lat6 = math.asin(math.sin(lat1)*math.cos(d/R) +
					math.cos(lat1)*math.sin(d/R)*math.cos(math.radians(bearing + (hbeam / 4))))

	lon6 = lon1 + math.atan2(math.sin(math.radians(bearing + (hbeam / 4)))*math.sin(d/R)*math.cos(lat1),
					math.cos(d/R)-math.sin(lat1)*math.sin(lat6))

	lat7 = math.asin(math.sin(lat1)*math.cos(d/R) +
					math.cos(lat1)*math.sin(d/R)*math.cos(math.radians(bearing + (3 * hbeam / 8))))

	lon7 = lon1 + math.atan2(math.sin(math.radians(bearing + (3 * hbeam / 8)))*math.sin(d/R)*math.cos(lat1),
					math.cos(d/R)-math.sin(lat1)*math.sin(lat7))

	lat8 = math.asin( math.sin(lat1)*math.cos(d/R) +
					math.cos(lat1)*math.sin(d/R)*math.cos(math.radians(bearing + (hbeam / 2))))

	lon8 = lon1 + math.atan2(math.sin(math.radians(bearing + (hbeam / 2)))*math.sin(d/R)*math.cos(lat1),
					math.cos(d/R)-math.sin(lat1)*math.sin(lat8))

	lat9 = math.asin( math.sin(lat1)*math.cos(d/R) +
					math.cos(lat1)*math.sin(d/R)*math.cos(math.radians(bearing + 180)))

	lon9 = lon1 + math.atan2(math.sin(math.radians(bearing + 180))*math.sin(d/R)*math.cos(lat1),
					math.cos(d/R)-math.sin(lat1)*math.sin(lat9))

	# RADIANS to DEGREES
	lat1 = math.degrees(lat1)
	lon1 = math.degrees(lon1)
	lat2 = math.degrees(lat2)
	lon2 = math.degrees(lon2)
	lat3 = math.degrees(lat3)
	lon3 = math.degrees(lon3)
	lat4 = math.degrees(lat4)
	lon4 = math.degrees(lon4)
	lat5 = math.degrees(lat5)
	lon5 = math.degrees(lon5)
	lat6 = math.degrees(lat6)
	lon6 = math.degrees(lon6)
	lat7 = math.degrees(lat7)
	lon7 = math.degrees(lon7)
	lat8 = math.degrees(lat8)
	lon8 = math.degrees(lon8)
	lat9 = math.degrees(lat9)
	lon9 = math.degrees(lon9)

	# If not an omni antenna
	if hbeam < 360:
		return "POLYGON (({} {}, {} {}, {} {}, {} {}, {} {}, {} {}, {} {}, {} {}, {} {}))".format(lon1, lat1, lon2, lat2, lon3, lat3, lon4, lat4, lon5, lat5, lon6, lat6, lon7, lat7, lon8, lat8, lon1, lat1)
	else:
		return "POLYGON (({} {}, {} {}, {} {}, {} {}, {} {}, {} {}, {} {}, {} {}, {} {}))".format(lon2, lat2, lon3, lat3, lon4, lat4, lon5, lat5, lon6, lat6, lon7, lat7, lon8, lat8, lon9, lat9, lon2, lat2)


In [4]:
gis_df = pd.read_csv("{}gis.csv".format(gis_path), names = ['Name','CILAC', 'SectorID', 'RNC_BSC', 'LAC', 'SectorType', 'Scr_Freq', \
                                                            'UARFCN', 'BSIC', 'Tech', 'Latitude', 'Longitude','Bearing','AvgNeighborDist', \
                                                            'MaxNeighborDist', 'NeighborsCount', 'Eng', 'TiltE','TiltM', 'SiteID', \
                                                            'AdminCellState', 'Asset', 'Asset_Configuration', 'Cell_Type', 'Cell_Name', \
                                                            'City', 'Height', 'RF_Team', 'Asset_Calc', 'Sector_uniq', 'FreqType', 'TAC', \
                                                            'RAC', 'Band', 'Vendor', 'CPICHPwr', 'MaxTransPwr', 'FreqMHz', 'HBW', \
                                                            'VBW', 'Antenna'])
gis_df['geometry'] = gis_df.apply(lambda x: polysTriangle(x['Latitude'], x['Longitude'], x['Bearing'], x['HBW']), axis = 1)
gis_df['geometry'] = gis_df['geometry'].apply(lambda x: shapely.wkt.loads(x))
gis_df = gpd.GeoDataFrame(gis_df, geometry='geometry', crs='EPSG:4326')
gis_df.head(2)

,Name,CILAC,SectorID,RNC_BSC,LAC,SectorType,Scr_Freq,UARFCN,BSIC,Tech,Latitude,Longitude,Bearing,AvgNeighborDist,MaxNeighborDist,NeighborsCount,Eng,TiltE,TiltM,SiteID,AdminCellState,Asset,Asset_Configuration,Cell_Type,Cell_Name,City,Height,RF_Team,Asset_Calc,Sector_uniq,FreqType,TAC,RAC,Band,Vendor,CPICHPwr,MaxTransPwr,FreqMHz,HBW,VBW,Antenna,geometry
0,KNTYS00368A_n70_AWS-4_UL5_1,41036398814,1036398814,-1,-1,401500,405,401500,NaN,NR_Dish,35.720944,-84.022500,0,NaN,NaN,NaN,TYS-13-KNFrndsville,2.0,0,35.7209_-84.0225,1,66392899,NaN,OUTDOOR,440 Dotson Memorial RD,Maryville,53.3400,KN,66392899,66392899#0,F1,25498.0,NaN,n70_AWS-4_UL5,NaN,NaN,45.07,2007.5,61.5,4.935897,KNTYS00368A,"POLYGON ((-84.0225 35.72094, -84.02533 35.7248..."
1,KNTYS00052A_n70_AWS-4_UL5_2,41036402722,1036402722,-1,-1,401500,706,401500,NaN,NR_Dish,35.754778,-83.932083,120,NaN,NaN,NaN,TYS-10-KNMaryville,2.0,0,35.7548_-83.9321,1,2120758710,NaN,OUTDOOR,1002A Grandview Drive,Maryville,24.6888,KN,2120758710,2120758710#120,F1,25498.0,NaN,n70_AWS-4_UL5,NaN,NaN,45.07,2007.5,61.5,4.935897,KNTYS00052A,"POLYGON ((-83.93208 35.75478, -83.92655 35.754..."


#### Load impacts - Data source for crossed feeders
#### Note: Grid-Cell data is not used for crossed feeders analysis as many location points are calculated using 1 cell only (meaning theyty are calculated using azimuth from GIS info) and therefore not suitable for crossed feeder analysis

In [5]:
impacts_df = pd.read_csv("{}denver-enriched-impacts.csv".format(input_path))
impacts_df.head(2)

,Unnamed: 0,index,tag,cilac,next_cilac,traffic_mobility,traffic_stationary,traffic_unknown,drop_mobility,drop_stationary,drop_unknown,voice_traffic_stationary,voice_traffic_mobility,voice_traffic_unknown,voice_drops_mobility,voice_drops_stationary,voice_drops_unknown,impact_time,drop_impact_time,cell_impact_name,cell_impact_tech,cell_impact_lat,cell_impact_lon,cell_impact_azimuth,cell_impact_elec_tilt,cell_impact_mech_tilt,cell_impact_site,cell_impact_market,cell_impact_band,cell_impact_vendor,cell_impact_hbw,cell_impact_tech_band,cell_impact_site_tech_band_list,cell_name,cell_tech,cell_lat,cell_lon,cell_azimuth,cell_elec_tilt,cell_mech_tilt,cell_site,cell_market,cell_band,cell_vendor,cell_hbw,cell_tech_band,cell_site_tech_band_list,co_site,co_sectored,traffic_data,traffic_voice,drops,drops_voice,total_cell_traffic_data,total_cell_traffic_voice,total_cell_traffic_data_gsm,total_cell_traffic_voice_gsm,total_cell_traffic_data_umts,total_cell_traffic_voice_umts,total_cell_traffic_data_lte,total_cell_traffic_voice_lte,total_cell_traffic_data_nsa,total_cell_traffic_voice_nsa,total_cell_traffic_data_sa,total_cell_traffic_voice_sa,relation_impact_data(%),relation_impact_voice(%),total_gsm_impact_data(%),total_umts_impact_data(%),total_lte_impact_data(%),total_nsa_impact_data(%),total_sa_impact_data(%),total_gsm_impact_voice(%),total_umts_impact_voice(%),total_lte_impact_voice(%),total_nsa_impact_voice(%),total_sa_impact_voice(%),total_cell_traffic_data_sa_n70_AWS-4_UL5,total_cell_traffic_voice_sa_n70_AWS-4_UL5,total_sa_n70_AWS-4_UL5_impact_data(%),total_sa_n70_AWS-4_UL5_impact_voice(%),total_cell_traffic_data_sa_n66_G,total_cell_traffic_voice_sa_n66_G,total_sa_n66_G_impact_data(%),total_sa_n66_G_impact_voice(%),total_cell_traffic_data_sa_n71_G,total_cell_traffic_voice_sa_n71_G,total_sa_n71_G_impact_data(%),total_sa_n71_G_impact_voice(%),total_cell_traffic_data_sa_n71_F-G,total_cell_traffic_voice_sa_n71_F-G,total_sa_n71_F-G_impact_data(%),total_sa_n71_F-G_impact_voice(%),total_cell_traffic_data_sa_n71_A,total_cell_traffic_voice_sa_n71_A,total_sa_n71_A_impact_data(%),total_sa_n71_A_impact_voice(%),total_cell_traffic_data_sa_n71_F,total_cell_traffic_voice_sa_n71_F,total_sa_n71_F_impact_data(%),total_sa_n71_F_impact_voice(%),distance,max_neigh_distance,median_neigh_distance,angle_cell_to_impact,angle_impact_to_cell,cell_azimuth_to_angle_diff,cell_impact_azimuth_to_angle_diff
0,0,97,-1,4585818566,4585818771,1,0,0,0,0,0,0,0,0,0,0,0,1,0,DNDEN00051D_n71_F-G_3,sa,39.731486,-104.930944,190.0,4.0,0.0,39.7315_-104.931,DN,n71_F-G,NaN,62.00631,sa_n71_F-G,"['sa_n29_E_DL', 'sa_n66_G', 'sa_n70_AWS-4_UL5'...",DNDEN00141B_n70_AWS-4_UL15_2,sa,39.743755,-104.965646,120.0,2.0,0.0,39.7438_-104.966,DN,n70_AWS-4_UL5,NaN,61.5,sa_n70_AWS-4_UL5,"['sa_n29_E_DL', 'sa_n66_G', 'sa_n70_AWS-4_UL5'...",n,n,1,0,0,0,10223,91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10223,91,0.01,0.0,0,0,0,0,100.0,0,0,0,0,100.0,10223.0,91.0,100.0,100.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,3266.873795,2443.300579,1227.495903,114.680410,294.702594,5.319590,104.702594
1,1,201,-1,4585818505,4585818708,0,1,0,0,0,0,0,0,0,0,0,0,11,0,DNDEN00163A_n71_F-G_3,sa,39.732209,-104.941152,240.0,3.0,0.0,39.7322_-104.941,DN,n71_F-G,NaN,63.00000,sa_n71_F-G,"['sa_n29_E_DL', 'sa_n66_G', 'sa_n70_AWS-4_UL5'...",DNDEN00124A_n66_G_1,sa,39.775778,-104.992833,0.0,2.0,0.0,39.7758_-104.993,DN,n66_G,NaN,66.0,sa_n66_G,"['sa_n29_E_DL', 'sa_n66_G', 'sa_n70_AWS-4_UL5'...",n,n,1,0,0,0,13281,115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13281,115,0.01,0.0,0,0,0,0,100.0,0,0,0,0,100.0,NaN,NaN,0.0,0.0,13281.0,115.0,100.0,100.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,6558.700006,4261.400195,3982.785140,137.620608,317.653657,137.620608,77.653657


In [6]:
print(impacts_df.columns.to_list())

['Unnamed: 0', 'index', 'tag', 'cilac', 'next_cilac', 'traffic_mobility', 'traffic_stationary', 'traffic_unknown', 'drop_mobility', 'drop_stationary', 'drop_unknown', 'voice_traffic_stationary', 'voice_traffic_mobility', 'voice_traffic_unknown', 'voice_drops_mobility', 'voice_drops_stationary', 'voice_drops_unknown', 'impact_time', 'drop_impact_time', 'cell_impact_name', 'cell_impact_tech', 'cell_impact_lat', 'cell_impact_lon', 'cell_impact_azimuth', 'cell_impact_elec_tilt', 'cell_impact_mech_tilt', 'cell_impact_site', 'cell_impact_market', 'cell_impact_band', 'cell_impact_vendor', 'cell_impact_hbw', 'cell_impact_tech_band', 'cell_impact_site_tech_band_list', 'cell_name', 'cell_tech', 'cell_lat', 'cell_lon', 'cell_azimuth', 'cell_elec_tilt', 'cell_mech_tilt', 'cell_site', 'cell_market', 'cell_band', 'cell_vendor', 'cell_hbw', 'cell_tech_band', 'cell_site_tech_band_list', 'co_site', 'co_sectored', 'traffic_data', 'traffic_voice', 'drops', 'drops_voice', 'total_cell_traffic_data', 'total

#### Calculate crossed-feeder score

In [7]:
import sys
import numpy as np

def calcScore(azimuth, hbw, distance, max_distance, median_distance, angle, relation_impact):
    # Exclude cases where BW >= 180 degrees (e.g. omni antenna) or distance == 0 etc.
    if (hbw >= 180) or (distance < 1000) or (max_distance < 1000) or np.isnan(azimuth) or np.isnan(angle):
        return 0.0

    # Limit the half-beamwidth used to 60°
    half_bw = min(hbw, 60)

    angle_1 = (360 + (azimuth - half_bw)) % 360
    angle_2 = (360 + (azimuth + half_bw)) % 360

    # Is angle inside the [angle_1, angle_2] sector?
    inside = ((angle_1 <= angle_2) and (angle_1 <= angle <= angle_2)) or \
             ((angle_2 < angle_1) and (angle >= angle_1 or angle <= angle_2))

    if inside:
        return 0.0

    # Outside sector: score based on distance & angular separation
    ang_diff = abs((angle - azimuth) % 360)
    ang_diff = min(ang_diff, 360 - ang_diff)  # 0..180
    return float(relation_impact) * min(distance / max_distance, 1.0) * (ang_diff / 180.0)

def classifyCrossedFeeders(cell_occurrences: int) -> str:
    if cell_occurrences >= 3:
        return "High potential: All feeders swapped"
    elif cell_occurrences == 2:
        return "High potential: 2 feeders swapped"
    else:
        return "Low potential: 1 feeder with high swapped score"

def calculateCrossedFeeders(impacts_df):
    # Score each row
    try:
        impacts_df['score'] = impacts_df.apply(lambda x: calcScore(x['cell_azimuth'],x['cell_hbw'], x['distance'], \
                                                                   x['max_neigh_distance'], x['median_neigh_distance'], \
                                                                   x['angle_cell_to_impact'], x['relation_impact_data(%)']), axis=1)
    except Exception as e:
        print(f"\tIssue calculating crossed feeder score: {e}")
        sys.exit(0)

    # Aggregate scores per cell and keep top 5% of by score
    try:
        tmp = impacts_df.assign(pos=(impacts_df['score'] > 0).astype('int8'))
        results = (
            tmp.groupby(['cell_site', 'cell_name', 'cell_band'], as_index=False)
               .agg(score=('score', 'sum'), scored_impacts=('pos', 'sum'))
        )
        if results.empty:
            return impacts_df, results

        # Limit to top 5% of scores per cell
        score_thresh = results['score'].quantile(0.95)
        results = results[results['score'] >= score_thresh].copy()

        # site_band key and per-site-band occurrence count (fast vectorized way)
        results['site_band'] = results['cell_site'].astype(str) + "_" + results['cell_band'].astype(str)
        results['cell_occurrences'] = results.groupby('site_band')['site_band'].transform('size')
        results['site_score'] = results.groupby('cell_site')['score'].transform('sum')
    except Exception as e:
        print(f"\tIssue manipulating crossed feeder results: {e}")
        sys.exit(0)

    # Classify cell-site
    try:
        results['classification'] = results['cell_occurrences'].apply(classifyCrossedFeeders)
    except Exception as e:
        print(f"\tIssue classifying crossed feeder results: {e}")
        sys.exit(0)

    return impacts_df, results

enriched_impacts_df_export, results = calculateCrossedFeeders(impacts_df)


In [8]:
print("There are {} potential crossed feeder cells".format(results.shape[0]))

There are 160 potential crossed feeder cells


#### Sort results by site score and cell_name

In [9]:
sorted_results = (
    results.sort_values(by=["site_score", "cell_name"], ascending=[False, True], na_position="last")
)
sorted_results.head(2)

,cell_site,cell_name,cell_band,score,scored_impacts,site_band,cell_occurrences,site_score,classification
826,39.5073_-104.879,DNDEN00416C_n66_G_2,n66_G,22.362910,42,39.5073_-104.879_n66_G,2,229.857348,High potential: 2 feeders swapped
827,39.5073_-104.879,DNDEN00416C_n66_G_3,n66_G,53.232947,26,39.5073_-104.879_n66_G,2,229.857348,High potential: 2 feeders swapped


#### For export lets clean up 'cell_site' and 'site_band'

In [10]:
sorted_results['cell_site'] = sorted_results.apply(lambda x: x.cell_name.split("_")[0], axis = 1)
sorted_results['site_band'] = sorted_results.apply(lambda x: x.cell_name.split("_")[0] + "_" + x.cell_band, axis = 1)
sorted_results.head()

,cell_site,cell_name,cell_band,score,scored_impacts,site_band,cell_occurrences,site_score,classification
826,DNDEN00416C,DNDEN00416C_n66_G_2,n66_G,22.362910,42,DNDEN00416C_n66_G,2,229.857348,High potential: 2 feeders swapped
827,DNDEN00416C,DNDEN00416C_n66_G_3,n66_G,53.232947,26,DNDEN00416C_n66_G,2,229.857348,High potential: 2 feeders swapped
829,DNDEN00416C,DNDEN00416C_n70_AWS-4_UL15_2,n70_AWS-4_UL5,23.959380,59,DNDEN00416C_n70_AWS-4_UL5,2,229.857348,High potential: 2 feeders swapped
830,DNDEN00416C,DNDEN00416C_n70_AWS-4_UL15_3,n70_AWS-4_UL5,45.984569,36,DNDEN00416C_n70_AWS-4_UL5,2,229.857348,High potential: 2 feeders swapped
832,DNDEN00416C,DNDEN00416C_n71_F-G_2,n71_F-G,28.644978,52,DNDEN00416C_n71_F-G,2,229.857348,High potential: 2 feeders swapped


In [81]:
sorted_results.head(20)

,cell_site,cell_name,cell_band,score,scored_impacts,site_band,cell_occurrences,site_score,classification
826,DNDEN00416C,DNDEN00416C_n66_G_2,n66_G,22.362910,42,DNDEN00416C_n66_G,2,229.857348,High potential: 2 feeders swapped
827,DNDEN00416C,DNDEN00416C_n66_G_3,n66_G,53.232947,26,DNDEN00416C_n66_G,2,229.857348,High potential: 2 feeders swapped
829,DNDEN00416C,DNDEN00416C_n70_AWS-4_UL15_2,n70_AWS-4_UL5,23.959380,59,DNDEN00416C_n70_AWS-4_UL5,2,229.857348,High potential: 2 feeders swapped
830,DNDEN00416C,DNDEN00416C_n70_AWS-4_UL15_3,n70_AWS-4_UL5,45.984569,36,DNDEN00416C_n70_AWS-4_UL5,2,229.857348,High potential: 2 feeders swapped
832,DNDEN00416C,DNDEN00416C_n71_F-G_2,n71_F-G,28.644978,52,DNDEN00416C_n71_F-G,2,229.857348,High potential: 2 feeders swapped
833,DNDEN00416C,DNDEN00416C_n71_F-G_3,n71_F-G,55.672563,51,DNDEN00416C_n71_F-G,2,229.857348,High potential: 2 feeders swapped
2411,DNDEN00234B,DNDEN00234B_n66_G_3,n66_G,31.190316,22,DNDEN00234B_n66_G,1,151.531952,Low potential: 1 feeder with high swapped score
2412,DNDEN00234B,DNDEN00234B_n70_AWS-4_UL15_1,n70_AWS-4_UL5,22.083592,9,DNDEN00234B_n70_AWS-4_UL5,3,151.531952,High potential: All feeders swapped
2413,DNDEN00234B,DNDEN00234B_n70_AWS-4_UL15_2,n70_AWS-4_UL5,20.268010,26,DNDEN00234B_n70_AWS-4_UL5,3,151.531952,High potential: All feeders swapped
2414,DNDEN00234B,DNDEN00234B_n70_AWS-4_UL15_3,n70_AWS-4_UL5,24.184249,29,DNDEN00234B_n70_AWS-4_UL5,3,151.531952,High potential: All feeders swapped


#### Check Examples for sanity

In [85]:
check_cell = "DNDEN00234B_n70_AWS-4_UL15_2"
impact_weight = 2

In [86]:
source_gpd = gis_df[gis_df.Name == check_cell].copy()
impact_gpd = gis_df[gis_df.Name.isin(impacts_df[(impacts_df.cell_name == check_cell) & \
                                     (impacts_df['relation_impact_data(%)'] >= impact_weight)].cell_impact_name.to_list())].copy()

In [87]:
m = impact_gpd.explore(color="red", name="Impact")
m = source_gpd.explore(m = m, color = "blue", name="Source")
folium.LayerControl().add_to(m)
m

## Co-Sectored Crosse Feeders

In [59]:
def calculateCoSectoredCrossedFeeders(impacts_df):
    # Reduce impacts_df to co-sited and different bands
    co_sectored_impacts_df = impacts_df[(impacts_df['cell_site'] == impacts_df['cell_impact_site']) & (impacts_df['cell_band'] != impacts_df['cell_impact_band'])].copy()

	# Create a cell mapping to identify co-sectors
    co_sectored_impacts_df['cell_co_sector'] = (co_sectored_impacts_df['cell_site'].astype(str) + "_" + co_sectored_impacts_df['cell_azimuth'].map(lambda x: f"{x:.0f}"))
    co_sectored_impacts_df['cell_impact_co_sector'] = (co_sectored_impacts_df['cell_impact_site'].astype(str) + "_" + co_sectored_impacts_df['cell_impact_azimuth'].map(lambda x: f"{x:.0f}"))
    
    # Split co_sectored_impacts_df into co_sector_impacts and non_co_sector_impacts
    co_sector_impact = co_sectored_impacts_df[co_sectored_impacts_df['cell_co_sector'] == co_sectored_impacts_df['cell_impact_co_sector']].copy()
    non_co_sector_impact = co_sectored_impacts_df[co_sectored_impacts_df['cell_co_sector'] != co_sectored_impacts_df['cell_impact_co_sector']].copy()
    
    co_sector_impact = co_sector_impact[['cell_site', 'cell_name', 'cell_band', 'cell_impact_name', 'cell_impact_band', \
                                         'relation_impact_data(%)', 'cell_azimuth', 'cell_impact_azimuth']].copy()
    
    co_sector_impact.rename(columns = {'cell_impact_name':'co_sectored_impact_cell', \
                                       'relation_impact_data(%)' : 'co_sectored_relation_impact(%)'}, inplace = True)
    
    non_co_sector_impact = non_co_sector_impact[['cell_name', 'cell_impact_name', 'cell_impact_band', \
                                                 'relation_impact_data(%)']].copy()
    
    non_co_sector_impact.rename(columns = {'cell_impact_name':'non_co_sectored_impact_cell', \
                                           'relation_impact_data(%)' : 'non_co_sectored_relation_impact(%)'}, inplace = True)

	# Merge into new df - final_co_sector_impacts
    final_co_sector_impacts = co_sector_impact.merge(non_co_sector_impact, on=['cell_name', 'cell_impact_band'], how='right')
    
    # Calculate percentage difference between co-sectored and non-cosectored, closer to -100% indicates crossed feeder
    co = final_co_sector_impacts['co_sectored_relation_impact(%)']
    non = final_co_sector_impacts['non_co_sectored_relation_impact(%)']
    denom = co + non
    final_co_sector_impacts['diff'] = np.where(denom != 0, 100.0 * (co - non) / denom, 0.0)
    
    # Remove any pair where 1 interaction is < 10% of overall transitions
    final_co_sector_impacts = final_co_sector_impacts[((final_co_sector_impacts['co_sectored_relation_impact(%)'] > 10) | (final_co_sector_impacts['non_co_sectored_relation_impact(%)'] > 10)) & (final_co_sector_impacts['diff'] < -10)].copy()
	
	# Count occurances of site-band-impact-band, only interested where this is >= 2
    num_cells = final_co_sector_impacts[['cell_site', 'cell_band', 'cell_impact_band', 'cell_name']].groupby(['cell_site', 'cell_band', 'cell_impact_band']).count()
    num_cells.rename(columns = {'cell_name':'cell_count'}, inplace = True)
    num_cells = num_cells[num_cells['cell_count'] > 1].copy()

	# Find combined 'diff' score
    combined_score = final_co_sector_impacts[['cell_site', 'cell_band', 'cell_impact_band', 'diff']].groupby(['cell_site', 'cell_band', 'cell_impact_band']).sum()
    combined_score.rename(columns = {'diff':'combined_diff'}, inplace = True)
    
    final_co_sector_impacts = final_co_sector_impacts.merge(num_cells, on=['cell_site', 'cell_band', 'cell_impact_band'], how='inner')
    final_co_sector_impacts = final_co_sector_impacts.merge(combined_score, on=['cell_site', 'cell_band', 'cell_impact_band'], how='inner').sort_values(by=['combined_diff', 'diff'])
    
    return final_co_sector_impacts

In [60]:
co_sectored_df = calculateCoSectoredCrossedFeeders(impacts_df)

In [61]:
co_sectored_df

,cell_site,cell_name,cell_band,co_sectored_impact_cell,cell_impact_band,co_sectored_relation_impact(%),cell_azimuth,cell_impact_azimuth,non_co_sectored_impact_cell,non_co_sectored_relation_impact(%),diff,cell_count,combined_diff
0,39.5077_-104.821,DNDEN00211C_n66_G_3,n66_G,DNDEN00211C_n71_F-G_3,n71_F-G,5.26,260.0,260.0,DNDEN00211C_n71_F-G_2,42.11,-77.791851,2,-111.167406
1,39.5077_-104.821,DNDEN00211C_n66_G_3,n66_G,DNDEN00211C_n71_F-G_3,n71_F-G,5.26,260.0,260.0,DNDEN00211C_n71_F-G_1,10.53,-33.375554,2,-111.167406
2,39.9871_-104.993,DNDEN00113A_n70_AWS-4_UL15_3,n70_AWS-4_UL5,DNDEN00113A_n71_F-G_3,n71_F-G,9.73,240.0,240.0,DNDEN00113A_n71_F-G_1,19.60,-33.651551,2,-63.001022
3,39.9871_-104.993,DNDEN00113A_n70_AWS-4_UL15_2,n70_AWS-4_UL5,DNDEN00113A_n71_F-G_2,n71_F-G,9.34,120.0,120.0,DNDEN00113A_n71_F-G_1,17.10,-29.349470,2,-63.001022


#### Chec crossed feeders

In [75]:
check_cell_1 = "DNDEN00211C_n66_G_3"
check_cell_2 = "DNDEN00211C_n71_F-G_1"
impact_weight = 2

In [76]:
source_gpd = gis_df[gis_df.Name == check_cell_1].copy()
impact_gpd_1 = gis_df[gis_df.Name.isin(impacts_df[(impacts_df.cell_name == check_cell_1) & \
                                     (impacts_df['relation_impact_data(%)'] >= impact_weight)].cell_impact_name.to_list())].copy()

impact_gpd_2 = gis_df[gis_df.Name.isin(impacts_df[(impacts_df.cell_name == check_cell_2) & \
                                     (impacts_df['relation_impact_data(%)'] >= impact_weight)].cell_impact_name.to_list())].copy()


In [77]:
m = impact_gpd_1.explore(color="red", name="Impact")
m = impact_gpd_2.explore(m = m, color="yellow", name="Impact")
m = source_gpd.explore(m = m, color = "blue", name="Source")
folium.LayerControl().add_to(m)
m

#### Save Results

In [14]:
sorted_results.to_csv("{}Crossed_Feeders_Results.csv".format(output_path))
enriched_impacts_df_export.to_csv("{}CrossedFeederImacts.csv".format(output_path))